In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [2]:
from ptranking.data.data_utils import LTRDataset, SPLIT_TYPE
data_id = 'MQ2007_Super'
#MQ2007内にあるもの
fold1 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S1.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold2 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S2.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold3 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S3.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold4 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S4.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold5 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S5.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
test_data = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='test.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)

In [3]:
layers = [nn.Linear(46, 128),
          nn.ReLU(),
          nn.Linear(128, 64),
          nn.ReLU(),
          nn.Linear(64, 32),
          nn.ReLU(),
          nn.Linear(32, 1)]

In [4]:
class RankNet(nn.Module):
    def __init__(self, layers):
        super(RankNet, self).__init__()
        self.model = nn.Sequential(*layers)
        
    def forward(self, batch_ranking, label):
        batch_pred = self.model(batch_ranking) # [1, 40, 1]
        batch_sij = torch.squeeze(batch_pred, 0) - torch.squeeze(batch_pred, 2) # [40, 40]

        label_dim = torch.squeeze(label)
        label_diffs = torch.unsqueeze(label_dim, 1) - label # [40,40]
        batch_Sij = torch.clamp(label_diffs, -1, 1)
                
        # computes p_ij
        batch_pij = 1 / (1 + torch.exp(-batch_sij)) # [40, 40]
                
        # computes loss
        batch_loss = 0.5 * (1-batch_Sij) * batch_sij + torch.log(1+torch.exp(-batch_sij)) #[40, 40]
                
        # 上三角行列
        batch_loss_triu = torch.triu(batch_loss, diagonal=1)

        # 損失の平均
        num_elements = (batch_loss_triu.size(1)*batch_loss_triu.size(1)-batch_loss_triu.size(1))/2
        batch_loss_mean = torch.sum(batch_loss)/num_elements
        
        #loss_list.append(batch_loss_mean)
                
        #print("batch_loss_mean", batch_loss_mean)
        
        return batch_loss_mean
    
    def predict(self, x):
        return self.model(x)

In [5]:
def DCG(sorted_labels, cutoff):
    denoms = torch.log2(torch.arange(2, cutoff+2))
    nums = torch.pow(2, sorted_labels[0:cutoff])-1
    dcg = sum(nums / denoms)
    return dcg

In [6]:
def nDCG(ideal, pred, k):
    dcg_f = DCG(pred, k)
    dcg = DCG(ideal, k)
    nDCG = dcg_f / dcg
    return nDCG

In [7]:
def compute_ndcg(rank,label,cutoff):     
    t = model.predict(rank)
    r = torch.argsort(t,dim=1,descending = True)
    ideal,_ = torch.sort(label,1,descending = True)
    ideal = torch.unsqueeze(ideal,2).reshape(-1)
    pred = torch.gather(label.unsqueeze(2),1,r).reshape(-1)
    ndcg = torch.nan_to_num(nDCG(ideal,pred,cutoff))
    return ndcg

In [8]:
def testing(data):
    with torch.no_grad():
        ndcg_sum = 0
        for _,batch_ranking,label in data:
        #computes ndcg
            ndcg = compute_ndcg(batch_ranking,label,5)
            ndcg_sum += ndcg  
    ndcg_sum /= len(data)
    return(ndcg_sum)

In [9]:
def training_loop(n_epochs,optimizer,model,data_list,learning_rate):
    count = 0
    while count < len(data_list):
        # 最初の3つをtrainに
        # 1つをvalidationに
        # 最後をtestに
        train = data_list[:3]
        validation = data_list[3]
        test = data_list[4]
        fold_train_loss = []
        fold_train_ndcg = []
        # fold loop
        for fold in train:
            loss_list = []
            ndcg_list = []
            # epoch 回の訓練
            for epoch in range(1, n_epochs + 1):
                test_ndcg_mean = []
                # クエリごとの訓練
                for _,batch_ranking,label in fold:

                    batch_loss_mean = model.forward(batch_ranking=batch_ranking, label=label)
                    loss_list.append(batch_loss_mean) # クエリ一つのloss
                    ndcg = compute_ndcg(batch_ranking,label,5)
                    ndcg_list.append(ndcg)

            fold_train_loss.append(sum(loss_list)/len(fold)) # fold1つあたりのloss
            fold_train_ndcg.append(sum(ndcg_list)/len(fold)) # fold1つあたりのndcg
        #validation
        val_ndcg = testing(validation)
        print("val", val_ndcg)
        #test
        test_ndcg = testing(test)
        print("test", test_ndcg)
        #　右にひとつシフト ([1,2,3,4,5] -> [5,1,2,3,4]となる)
        data_list.insert(0,data_list.pop())
        
        # fold の平均
        epoch_loss = sum(fold_train_loss)/len(fold_train_loss)
        epoch_ndcg = sum(fold_train_ndcg)/len(fold_train_ndcg)
        
#         if epoch%10==0:
#             print("epoch: ", epoch, "loss: ", epoch_loss, "ndcg: ", epoch_ndcg)
        count += 1
        learning_rate *= 10
        optimizer.zero_grad()
        batch_loss_mean.backward()
        optimizer.step()

In [10]:
data_list = [fold1,fold2,fold3,fold4,fold5]

In [11]:
# クラスのインスタンス化
model = RankNet(layers)
learning_rate = 0.001

In [12]:
training_loop(
    n_epochs = 1,
    optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate),
    model = model,
    data_list = data_list,
    learning_rate = 0.001
    )

val tensor(0.2567)
test tensor(0.2485)
val tensor(0.2270)
test tensor(0.2568)
val tensor(0.2317)
test tensor(0.2272)
val tensor(0.2396)
test tensor(0.2318)
val tensor(0.2484)
test tensor(0.2395)
